# Question Answering – Preprocessing des données

Ce notebook a pour objectif de préparer les données du dataset SQuAD
pour l’entraînement d’un modèle Transformer en question answering
extractif.


## Objectifs

- Tokeniser les paires (question, contexte)
- Gérer les contextes longs par découpage
- Aligner les positions de début et de fin de la réponse
  avec les tokens générés par le tokenizer



In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np


c:\Users\aissi\OneDrive - De Vinci\A5\UVSQ\S2\Fouille de données\qa-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Chargement du dataset


In [2]:
dataset = load_dataset("squad")

## Initialisation du tokenizer


In [3]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## Paramètres de tokenization

Les contextes pouvant être longs, nous utilisons :
- une longueur maximale
- un stride pour le découpage


In [4]:
max_length = 384
doc_stride = 128

## Fonction de preprocessing

Cette fonction permet :
- de tokeniser les données
- de gérer les contextes longs
- d’aligner les positions des réponses avec les tokens


In [5]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]

    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_mapping = inputs.pop("overflow_to_sample_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = inputs["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = inputs.sequence_ids(i)
        sample_index = sample_mapping[i]

        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and
                    offsets[token_end_index][1] >= end_char):
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_positions.append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_positions.append(token_end_index + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs


## Application du preprocessing au dataset


In [6]:
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

Map: 100%|██████████| 10570/10570 [00:05<00:00, 1834.18 examples/s]


## Vérification des données préprocessées


In [7]:
tokenized_datasets["train"][0]

{'input_ids': [101,
  2000,
  3183,
  2106,
  1996,
  6261,
  2984,
  9382,
  3711,
  1999,
  8517,
  1999,
  10223,
  26371,
  2605,
  1029,
  102,
  6549,
  2135,
  1010,
  1996,
  2082,
  2038,
  1037,
  3234,
  2839,
  1012,
  10234,
  1996,
  2364,
  2311,
  1005,
  1055,
  2751,
  8514,
  2003,
  1037,
  3585,
  6231,
  1997,
  1996,
  6261,
  2984,
  1012,
  3202,
  1999,
  2392,
  1997,
  1996,
  2364,
  2311,
  1998,
  5307,
  2009,
  1010,
  2003,
  1037,
  6967,
  6231,
  1997,
  4828,
  2007,
  2608,
  2039,
  14995,
  6924,
  2007,
  1996,
  5722,
  1000,
  2310,
  3490,
  2618,
  4748,
  2033,
  18168,
  5267,
  1000,
  1012,
  2279,
  2000,
  1996,
  2364,
  2311,
  2003,
  1996,
  13546,
  1997,
  1996,
  6730,
  2540,
  1012,
  3202,
  2369,
  1996,
  13546,
  2003,
  1996,
  24665,
  23052,
  1010,
  1037,
  14042,
  2173,
  1997,
  7083,
  1998,
  9185,
  1012,
  2009,
  2003,
  1037,
  15059,
  1997,
  1996,
  24665,
  23052,
  2012,
  10223,
  26371,
  1010,
  2605

## Sauvegarde des données préprocessées

Les données tokenisées sont sauvegardées afin d’être réutilisées
directement lors de l’entraînement et de l’évaluation.


In [ ]:
tokenized_datasets.save_to_disk("outputs/tokenized_squad")


## Conclusion

Les données ont été correctement tokenisées et les positions des réponses
ont été alignées avec les tokens.

Ces données peuvent maintenant être utilisées pour l’entraînement
d’un modèle Transformer en question answering extractif.
